In [9]:
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, StringType, IntegerType , DoubleType
  
spark = SparkSession.builder\
        .master("local")\
        .appName("walmart sales")\
        .getOrCreate()

customer_schema = StructType([ 
    
    StructField('CustomerId', 
                IntegerType(), True), 
    StructField('Name', 
                StringType(), True), 
    StructField('City', 
                StringType(), True), 
    StructField('State', 
                StringType(), True), 
    StructField('ZipCode', 
                IntegerType(), True) 
]) 
sales_schema = StructType([ 
    
    StructField('SalesTxnId', 
                IntegerType(), True), 
    StructField('CategoryId', 
                IntegerType(), True),  
    StructField('CategoryName', 
                StringType(), True),
    StructField('ProductId', 
                IntegerType(), True), 
    StructField('ProductName', 
                StringType(), True), 
    StructField('Price', 
                DoubleType(), True), 
    StructField('Quantity', 
                IntegerType(), True),
    StructField('CustomerId', 
                IntegerType(), True) 
]) 
#Read tsv file 
customer_df = spark.read\
    .format("csv")\
    .schema(customer_schema)\
        .options(header = "False", sep = "\t")\
            .load(r"your file location path")
sales_df = spark.read\
    .format("csv")\
    .schema(sales_schema)\
        .options(header = "False", sep = "\t" )\
            .load(r"your file location path")

customer_df.show(5)
sales_df.show(5)

+----------+--------------+---------+-----+-------+
|CustomerId|          Name|     City|State|ZipCode|
+----------+--------------+---------+-----+-------+
|     11039|   Mary Torres|   Caguas|   PR|    725|
|      5623|    Jose Haley| Columbus|   OH|  43207|
|      5829|    Mary Smith|  Houston|   TX|  77015|
|      6336|Richard Maddox|   Caguas|   PR|    725|
|      1708|Margaret Booth|Arlington|   TX|  76010|
+----------+--------------+---------+-----+-------+
only showing top 5 rows

+----------+----------+----------------+---------+--------------------+------+--------+----------+
|SalesTxnId|CategoryId|    CategoryName|ProductId|         ProductName| Price|Quantity|CustomerId|
+----------+----------+----------------+---------+--------------------+------+--------+----------+
|         1|        43|Camping & Hiking|      957|Diamondback Women...|299.98|       1|     11599|
|         2|        48|    Water Sports|     1073|Pelican Sunstream...|199.99|       1|       256|
|         3|

1.Total Number of Customers:
How many unique customers are there in the dataset?

In [10]:
sales_df.createOrReplaceTempView("sales")
customer_df.createOrReplaceTempView("customers")

spark.sql("""select count(customerid) as TotalUniqueCustomers from (select distinct customerid
from customers
union 
select distinct customerid
from sales) """).show()

+--------------------+
|TotalUniqueCustomers|
+--------------------+
|               12346|
+--------------------+



2.Total Sales by State:
What is the total sales amount for each state?

In [11]:
sales_df.createOrReplaceTempView("salse")
customer_df.createOrReplaceTempView("customers")
spark.sql("""
select 
     c.state,cast(sum(sales.AggrigatedSale) as decimal(10,2)) as TotalSales     
from 
customers c
 join
(select 
   cast(sum(s.price * s.quantity) as decimal(10,2)) as AggrigatedSale,
   s.customerid
from 
salse s
group by s.customerid 
) as sales on c.customerid = sales.customerid group by c.state
          order by c.state
          """).show()

+-----+----------+
|state|TotalSales|
+-----+----------+
|   AR|   2136.67|
|   AZ|  48702.68|
|   CA| 503205.49|
|   CO|  40321.13|
|   CT|  19206.77|
|   DC|   8798.76|
|   DE|   1305.76|
|   FL|  93359.15|
|   GA|  38056.33|
|   HI|  35682.81|
|   ID|  10098.95|
|   IL| 116223.17|
|   IN|   6963.97|
|   KS|   2999.71|
|   KY|   2749.70|
|   LA|  24449.42|
|   MA|  29039.35|
|   MD|  51982.49|
|   MI|  83347.09|
|   MN|   3549.60|
+-----+----------+
only showing top 20 rows



3.Top 10 Most Purchased Products:
Which are the top 10 most purchased products based on the quantity sold?

In [12]:
sales_df.createOrReplaceTempView("salse")

spark.sql("""
select 
ProductName as Top_10_Products,
sum(quantity) as Quantity           
from 
salse
group by ProductName
ORDER BY Quantity desc limit 10
""").show()

+--------------------+--------+
|     Top_10_Products|Quantity|
+--------------------+--------+
|Perfect Fitness P...|   73698|
|Nike Men's Dri-FI...|   62956|
|O'Brien Men's Neo...|   57803|
|Nike Men's Free 5...|   36680|
|Under Armour Girl...|   31735|
|Nike Men's CJ Eli...|   22246|
|Field & Stream Sp...|   17325|
|Pelican Sunstream...|   15500|
|Diamondback Women...|   13729|
|ENO Atlas Hammock...|     998|
+--------------------+--------+



4.Average Transaction Value:
What is the average price of transactions across all sales?

In [13]:
sales_df.createOrReplaceTempView("sales")

spark.sql("""
select  cast(avg(Totalprice) as decimal(10,2)) as AverageTransactionPrice  from (
          select 
SalesTxnId , cast(sum(price * quantity) as decimal(10,2))  as TotalPrice 
from     
sales 
group by SalesTxnId) as TotalTransation 
""").show()


+-----------------------+
|AverageTransactionPrice|
+-----------------------+
|                 199.32|
+-----------------------+



5.Top 5 Customers by Expenditure:
Who are the top 5 customers by total amount spent?

In [14]:
sales_df.createOrReplaceTempView("sales")
customer_df.createOrReplaceTempView("customers")
spark.sql("""
select 
        s.customerId,  c.Name , sum(cast(s.price * s.Quantity as decimal(10,2)) ) as TotalSpent 
from 
customers c
join 
sales s
on
c.customerid = s.customerid
group by s.customerid,c.name
order by TotalSpent desc limit 5
""").show()

+----------+-----------------+----------+
|customerId|             Name|TotalSpent|
+----------+-----------------+----------+
|      9371|   Mary Patterson|   9299.03|
|       664|    Bobby Jimenez|   8394.26|
|     12431|        Mary Rios|   8073.15|
|     10591| Deborah Humphrey|   7889.05|
|      9271|Christopher Smith|   7665.25|
+----------+-----------------+----------+



6.Product Purchases by a Specific Customer:
List all products purchased by a specific customer (e.g., customer with ID 256), including the product name, quantity, and total amount spent.

In [15]:
sales_df.createOrReplaceTempView("sales")
customer_df.createOrReplaceTempView("customers")
spark.sql(
"""
select 
s.customerid,s.productname,sum(s.quantity) as TotalQuantity,sum(cast(s.price * s.Quantity as decimal(10,2)) ) as TotalSpent  
from
customers c
join 
sales s
on c.customerid = s.customerid
where c.customerid = 664
group by s.customerid , c.name , s.productname 
"""
).show()

+----------+--------------------+-------------+----------+
|customerid|         productname|TotalQuantity|TotalSpent|
+----------+--------------------+-------------+----------+
|       664|Nike Women's Temp...|            5|    150.00|
|       664|O'Brien Men's Neo...|            4|    199.92|
|       664|Field & Stream Sp...|            8|   3199.84|
|       664|Columbia Men's PF...|            3|     90.00|
|       664|Glove It Women's ...|            5|     99.95|
|       664|Diamondback Women...|            4|   1199.92|
|       664|Team Golf San Fra...|            5|    124.95|
|       664|Nike Men's Dri-FI...|           10|    500.00|
|       664|Pelican Sunstream...|            3|    599.97|
|       664|Perfect Fitness P...|           22|   1319.78|
|       664|Nike Men's CJ Eli...|            7|    909.93|
+----------+--------------------+-------------+----------+



7.Monthly Sales Trends:
Assuming there is a date field, analyze the sales trends over the months. Which month had the highest sales?

In [ ]:
sales_df.createOrReplaceTempView("sales")
spark.sql("""
select  Month , Totalsales  from 
(select  cast(sum(price * quantity)as decimal(10,2)) as Totalsales, month(date) as Month from
sales groupby Month) as TotalMonthSale 
order by
Totalsales desc limit 1
""").show()

8.Category with Highest Sales:
Which product category generated the highest total sales revenue?

In [24]:
sales_df.createOrReplaceTempView("sales")
spark.sql("""
select CategoryName , cast(sum(price * quantity) as decimal(10,2)) as TotalSales
from 
sales
group by CategoryName
order by TotalSales desc limit 1
""").show()

+------------+----------+
|CategoryName|TotalSales|
+------------+----------+
|     Fishing|6929653.50|
+------------+----------+



9.State-wise Sales Comparison:
Compare the total sales between two specific states (e.g., Texas vs. Ohio). Which state had higher sales?

In [25]:
sales_df.createOrReplaceTempView("sales")
customer_df.createOrReplaceTempView("customers")
spark.sql("""
          select *, rank() over(order by TotalSales desc) as HighestSales from 
          (select * from 
(select  c.State  , sum(cast(s.Price * s.Quantity as decimal(10,2))) as TotalSales from  sales s join customers c on s.CustomerId = c.CustomerId  
          group by c.state ) as statesale where State in ('DC','MN')) as higherSales 
""").show()

+-----+----------+------------+
|State|TotalSales|HighestSales|
+-----+----------+------------+
|   DC|   8798.76|           1|
|   MN|   3549.60|           2|
+-----+----------+------------+



10.Detailed Customer Purchase Report:
Generate a detailed report showing each customer along with their total purchases, the total number of transactions they have made, and the average transaction value.

In [26]:
sales_df.createOrReplaceTempView("sales")
customer_df.createOrReplaceTempView("customers")
spark.sql("""
select * , cast((TotalPurchases / TotalTransation) as decimal(10,2))  as AverageTransaction   from (
select  c.CustomerId ,count(s.SalesTxnId) as TotalTransation , sum(cast(s.Price * s.Quantity as decimal(10,2))) as TotalPurchases 
from  sales s 
join customers c on s.CustomerId = c.CustomerId  
group by c.CustomerId ) as TransationTable  group by CustomerId,TotalTransation,TotalPurchases order by TotalTransation,CustomerId desc
""").show()

+----------+---------------+--------------+------------------+
|CustomerId|TotalTransation|TotalPurchases|AverageTransaction|
+----------+---------------+--------------+------------------+
|     10054|              1|         59.99|             59.99|
|      8985|              1|        399.98|            399.98|
|      7909|              1|         99.99|             99.99|
|      7091|              1|         49.95|             49.95|
|      6372|              1|         74.97|             74.97|
|      4158|              1|        199.99|            199.99|
|      3278|              1|        129.99|            129.99|
|      1476|              1|        129.99|            129.99|
|     11347|              2|        189.98|             94.99|
|     10465|              2|        249.98|            124.99|
|     10243|              2|        329.98|            164.99|
|      8798|              2|        599.97|            299.99|
|      8783|              2|        519.95|            